# Udacity WeRateDogs Data Wrangling Project

## Introduction
This is just some text to remind me to write a introduction for the project

In [1]:
import requests
import os
import json
import time
import tweepy
import pandas as pd

## Data Wrangling

In this section of the report I will gather the necessary data, understand its general properties, identify and clean possible quality and tidiness errors such as missing or incorrect values.

### Gather

Here I will be gathering each of the three pieces of data for this project.  
  
1. **WeRateDogs Twitter archive:** this file is provided by Udacity in a `.csv` file called `twitter_archive_enhanced.csv`.
2. **Image predictions:** has information about the breed of dog or object shown in the tweet photo. It is stored on Udacity's servers in the file `image_predictions.tsv`.
3. **Additional data about each tweet**: information such as the number of likes or retweets of each tweet. It can be accessed using the Twitter API with tweepy.

#### WeRateDogs Twitter Archive

In [2]:
# Converting csv file to Pandas DataFrame
twitter_archive_df = pd.read_csv('data/twitter-archive-enhanced.csv')
twitter_archive_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


#### Image Predictions

In [3]:
# Getting the file from the Udacity server and saving it in the data folder
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open(os.path.join('data', url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

In [4]:
# Converting csv file to Pandas DataFrame
image_predictions_df = pd.read_csv('data/image-predictions.tsv', sep='\t')
image_predictions_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


#### Additional data about each tweet

In [5]:
# Keys and tokens provided by Twitter
consumer_key = 'YOUR CONSUMER KEY HERE'
consumer_secret = 'YOUR CONSUMER SECRET HERE'
access_token = 'YOUR ACCESS TOKEN HERE'
access_secret = 'YOUR ACCESS SECRET HERE'

# Creating Twitter API object with rate limits parameters
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [6]:
# Creating empty list to store tweet data (json format)
tweet_json_list = []
# Creating empty dict to store the tweets that can't be accessed
errors_dict = {}
# Defining the start time to check how long it took to access the data
start_time = time.time()

for tweet_id in twitter_archive_df.tweet_id.values:
    try:
        # Adding tweet info to tweet_json_list
        tweet = api.get_status(tweet_id, tweet_mode = 'extended')
        tweet_json_list.append(tweet._json)
    except tweepy.TweepError as error:
        # Adding tweets that could't be accessed in the errors_dict
        errors_dict[tweet_id] = error
            
# Checking how much time the was spent
elapsed_time = time.time() - start_time

# Printing elapsed time in HH:MM:SS format
hms_elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print('\n Time elapsed (HH:MM:SS): ' + hms_elapsed_time)
print('-'*55)

# Printing each tweet_id and error in the errors_dict
for tweet_id in errors_dict:
    print('ID:', tweet_id, 'ERROR:', errors_dict[tweet_id])
print('-'*55)

# Checking number of errors
print('\n Number of errors: ' + str(len(errors_dict)))


 Time elapsed (HH:MM:SS): 00:45:45
-------------------------------------------------------
ID: 888202515573088257 ERROR: [{'code': 144, 'message': 'No status found with that ID.'}]
ID: 873697596434513921 ERROR: [{'code': 144, 'message': 'No status found with that ID.'}]
ID: 872668790621863937 ERROR: [{'code': 144, 'message': 'No status found with that ID.'}]
ID: 872261713294495745 ERROR: [{'code': 144, 'message': 'No status found with that ID.'}]
ID: 869988702071779329 ERROR: [{'code': 144, 'message': 'No status found with that ID.'}]
ID: 866816280283807744 ERROR: [{'code': 144, 'message': 'No status found with that ID.'}]
ID: 861769973181624320 ERROR: [{'code': 144, 'message': 'No status found with that ID.'}]
ID: 856602993587888130 ERROR: [{'code': 144, 'message': 'No status found with that ID.'}]
ID: 851953902622658560 ERROR: [{'code': 144, 'message': 'No status found with that ID.'}]
ID: 845459076796616705 ERROR: [{'code': 144, 'message': 'No status found with that ID.'}]
ID: 8447

In [7]:
# Writting tweet data to a .txt file
with open('data/tweet-json.txt', 'w') as outfile:
    for tweet_json in tweet_json_list:
        json.dump(tweet_json, outfile)
        outfile.write('\n')

In [8]:
# Creating empty list that will be used to hold fav and rt count for each tweet
tweet_extra_data_list = []

# Reading .txt file 
with open('data/tweet-json.txt', 'r') as json_file:
    # Reading first line
    line = json_file.readline()
    
    # While there's a next line execute following code
    while line:
        # Select tweet id, fav and rt count
        tweet = json.loads(line)
        tweet_id = tweet['id']
        tweet_retweet_count = tweet['retweet_count']
        tweet_favorite_count = tweet['favorite_count']
        
        # Save selected data to a dict
        tweet_data = {'tweet_id': tweet_id, 
                      'retweet_count': tweet_retweet_count, 
                      'favorite_count': tweet_favorite_count,
                     }
        
        # Add tweet_data to tweet_json_data
        tweet_extra_data_list.append(tweet_data)

        # Read next line
        line = json_file.readline()

tweet_extra_data_list[0]

{'tweet_id': 892420643555336193,
 'retweet_count': 7725,
 'favorite_count': 36279}

In [9]:
# Creating Pandas DataFrame with tweet_json_data
tweet_extra_data_df = pd.DataFrame(tweet_extra_data_list, 
                                   columns = ['tweet_id',
                                              'retweet_count',
                                              'favorite_count'])

# Creating a csv file with the extra data
tweet_extra_data_df.to_csv('data/tweet_extra_data.csv', index=False)


tweet_extra_data_df.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7725,36279
1,892177421306343426,5710,31278
2,891815181378084864,3785,23542
3,891689557279858688,7878,39567
4,891327558926688256,8496,37784


### Assess
The main objective of the Access part is to better understand each piece of data and identify possible issues that must be cleaned.  
All identified issues will be listed after accessing the data and divided between quality and tidiness problems for each piece of data, in order to facilitate the reading of the report.

#### WeRateDogs Twitter Archive

In [10]:
# Visualizing 5 random rows
twitter_archive_df.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2051,671488513339211776,NaN,NaN,2015-12-01 00:38:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Julius. He's a cool dog. Carries seash...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/671488513...,8,10,Julius,None,None,None,None
1449,696100768806522880,NaN,NaN,2016-02-06 22:38:50 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This poor pupper has been stuck in a vortex si...,NaN,NaN,NaN,https://vine.co/v/i1KWj0vbvA9,10,10,None,None,None,pupper,None
1973,673213039743795200,NaN,NaN,2015-12-05 18:51:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Dexter. He just got some big news. 10/...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/673213039...,10,10,Dexter,None,None,None,None
98,873213775632977920,NaN,NaN,2017-06-09 16:22:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sierra. She's one precious pupper. Abs...,NaN,NaN,NaN,https://www.gofundme.com/help-my-baby-sierra-g...,12,10,Sierra,None,None,pupper,None
2044,671533943490011136,NaN,NaN,2015-12-01 03:39:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Super rare dog here. Spiffy mohawk. Sharp mout...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/671533943...,6,10,None,None,None,None,None


In [11]:
# Checking df number of rows and columns
twitter_archive_df.shape

(2356, 17)

In [12]:
# Checking df datatypes
twitter_archive_df.dtypes

tweet_id                        int64
in_reply_to_status_id         float64
in_reply_to_user_id           float64
timestamp                      object
source                         object
text                           object
retweeted_status_id           float64
retweeted_status_user_id      float64
retweeted_status_timestamp     object
expanded_urls                  object
rating_numerator                int64
rating_denominator              int64
name                           object
doggo                          object
floofer                        object
pupper                         object
puppo                          object
dtype: object

In [13]:
# Checking missing values
twitter_archive_df.isnull().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [14]:
# I found it strange that there were None values in some columns and wanted to understand why
twitter_archive_df.query('name == "None"').head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
12,889665388333682689,NaN,NaN,2017-07-25 01:55:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a puppo that seems to be on the fence a...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/889665388...,13,10,None,None,None,None,puppo
24,887343217045368832,NaN,NaN,2017-07-18 16:08:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",You may not have known you needed to see this ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887343217...,13,10,None,None,None,None,None
25,887101392804085760,NaN,NaN,2017-07-18 00:07:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This... is a Jubilant Antarctic House Bear. We...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887101392...,12,10,None,None,None,None,None


In [15]:
# Checking name column values to see if there's any inconsistency
twitter_archive_df.name.unique()

array(['Phineas', 'Tilly', 'Archie', 'Darla', 'Franklin', 'None', 'Jax',
       'Zoey', 'Cassie', 'Koda', 'Bruno', 'Ted', 'Stuart', 'Oliver',
       'Jim', 'Zeke', 'Ralphus', 'Canela', 'Gerald', 'Jeffrey', 'such',
       'Maya', 'Mingus', 'Derek', 'Roscoe', 'Waffles', 'Jimbo', 'Maisey',
       'Lilly', 'Earl', 'Lola', 'Kevin', 'Yogi', 'Noah', 'Bella',
       'Grizzwald', 'Rusty', 'Gus', 'Stanley', 'Alfy', 'Koko', 'Rey',
       'Gary', 'a', 'Elliot', 'Louis', 'Jesse', 'Romeo', 'Bailey',
       'Duddles', 'Jack', 'Emmy', 'Steven', 'Beau', 'Snoopy', 'Shadow',
       'Terrance', 'Aja', 'Penny', 'Dante', 'Nelly', 'Ginger', 'Benedict',
       'Venti', 'Goose', 'Nugget', 'Cash', 'Coco', 'Jed', 'Sebastian',
       'Walter', 'Sierra', 'Monkey', 'Harry', 'Kody', 'Lassie', 'Rover',
       'Napolean', 'Dawn', 'Boomer', 'Cody', 'Rumble', 'Clifford',
       'quite', 'Dewey', 'Scout', 'Gizmo', 'Cooper', 'Harold', 'Shikha',
       'Jamesy', 'Lili', 'Sammy', 'Meatball', 'Paisley', 'Albus',
       'Nept

In [16]:
# Looking at the unique sources
twitter_archive_df.source.unique()

array(['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'],
      dtype=object)

In [17]:
# Trying to understand what was this 'Vine - Make a Scene' value
twitter_archive_df[twitter_archive_df.source == 
                   '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>'].head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
529,808344865868283904,NaN,NaN,2016-12-12 16:16:49 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Seamus. He's very bad at entering pool...,NaN,NaN,NaN,https://vine.co/v/5QWd3LZqXxd,11,10,Seamus,None,None,None,None


In [18]:
# Selecting replies that have urls so I can look at the tweet
twitter_archive_df[twitter_archive_df.in_reply_to_status_id.notnull() &
                   twitter_archive_df.expanded_urls.notnull()].head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
149,863079547188785154,6.671522e+17,4.196984e+09,2017-05-12 17:12:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Ladies and gentlemen... I found Pipsy. He may ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/863079547...,14,10,None,None,None,None,None
184,856526610513747968,8.558181e+17,4.196984e+09,2017-04-24 15:13:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...","THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/856526610...,14,10,None,None,None,None,None
251,844979544864018432,7.590995e+17,4.196984e+09,2017-03-23 18:29:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",PUPDATE: I'm proud to announce that Toby is 23...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/844979544...,13,10,None,None,None,None,None
565,802265048156610565,7.331095e+17,4.196984e+09,2016-11-25 21:37:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Like doggo, like pupper version 2. Both 11/10 ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/802265048...,11,10,None,doggo,None,pupper,None
1016,746906459439529985,7.468859e+17,4.196984e+09,2016-06-26 03:22:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...","PUPDATE: can't see any. Even if I could, I cou...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/746906459...,0,10,None,None,None,None,None


In [19]:
# Selecting a tweet to look
twitter_archive_df.iloc[251].expanded_urls.split(',')

['https://twitter.com/dog_rates/status/844979544864018432/photo/1',
 'https://twitter.com/dog_rates/status/844979544864018432/photo/1',
 'https://twitter.com/dog_rates/status/844979544864018432/photo/1']

In [20]:
# Selecting retweets that have urls so I can look at the tweet
twitter_archive_df[twitter_archive_df.retweeted_status_id.notnull() &
                   twitter_archive_df.expanded_urls.notnull()].head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,1.960740e+07,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Lilly. She just paralle...,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,https://twitter.com/dog_rates/status/830583320...,13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Emmy. She was adopted t...,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,https://twitter.com/dog_rates/status/878057613...,14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Shadow. In an attempt to r...,8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None


In [21]:
# Selecting a tweet to look
twitter_archive_df.iloc[32].expanded_urls.split(',')

['https://twitter.com/dog_rates/status/886053434075471873',
 'https://twitter.com/dog_rates/status/886053434075471873']

In [22]:
# Checking doggo values that aren't "None"
twitter_archive_df.query('doggo != "None"').head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None
43,884162670584377345,NaN,NaN,2017-07-09 21:29:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Yogi. He doesn't have any important dog m...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/884162670...,12,10,Yogi,doggo,None,None,None
99,872967104147763200,NaN,NaN,2017-06-09 00:02:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a very large dog. He has a date later. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/872967104...,12,10,None,doggo,None,None,None
108,871515927908634625,NaN,NaN,2017-06-04 23:56:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Napolean. He's a Raggedy East Nicaragu...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/871515927...,12,10,Napolean,doggo,None,None,None
110,871102520638267392,NaN,NaN,2017-06-03 20:33:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Never doubt a doggo 14/10 https://t.co/AbBLh2FZCH,NaN,NaN,NaN,https://twitter.com/animalcog/status/871075758...,14,10,None,doggo,None,None,None


#### Image Predictions

In [23]:
# Visualizing 5 random rows
image_predictions_df.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1819,834209720923721728,https://pbs.twimg.com/media/C5O1UAaWIAAMBMd.jpg,1,golden_retriever,0.754799,True,Pekinese,0.197861,True,Labrador_retriever,0.008654,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
1865,843235543001513987,https://pbs.twimg.com/media/C7PGQJAWwAAibui.jpg,1,Pembroke,0.958452,True,Cardigan,0.023770,True,Chihuahua,0.005269,True
425,674075285688614912,https://pbs.twimg.com/media/CVrL5YBWoAA_uPD.jpg,1,Airedale,0.305392,True,Kerry_blue_terrier,0.250014,True,Lakeland_terrier,0.188668,True
1568,794332329137291264,https://pbs.twimg.com/media/CwYJBiHXgAQlvrh.jpg,1,Samoyed,0.988307,True,malamute,0.004906,True,Great_Pyrenees,0.002901,True


In [24]:
# Checking image_predictions shape
image_predictions_df.shape

(2075, 12)

In [25]:
# Checking columns data types
image_predictions_df.dtypes

tweet_id      int64
jpg_url      object
img_num       int64
p1           object
p1_conf     float64
p1_dog         bool
p2           object
p2_conf     float64
p2_dog         bool
p3           object
p3_conf     float64
p3_dog         bool
dtype: object

In [26]:
# Looking for missing values
image_predictions_df.isnull().sum()

tweet_id    0
jpg_url     0
img_num     0
p1          0
p1_conf     0
p1_dog      0
p2          0
p2_conf     0
p2_dog      0
p3          0
p3_conf     0
p3_dog      0
dtype: int64

In [27]:
# Checking some photos to better understand the predictions
print(image_predictions_df.iloc[1037].jpg_url)
print(image_predictions_df.iloc[442].jpg_url)
print(image_predictions_df.iloc[293].jpg_url)
print(image_predictions_df.iloc[1480].jpg_url)

https://pbs.twimg.com/tweet_video_thumb/CeGGkWuUUAAYWU1.jpg
https://pbs.twimg.com/media/CVzG3yOVAAAqi9I.jpg
https://pbs.twimg.com/media/CVEbFDRWsAAkN_7.jpg
https://pbs.twimg.com/media/CtYqeNHWgAATqYZ.jpg


In [28]:
# Looking for inconsistencies in p1 values
image_predictions_df.p1.unique()[:30]

array(['Welsh_springer_spaniel', 'redbone', 'German_shepherd',
       'Rhodesian_ridgeback', 'miniature_pinscher',
       'Bernese_mountain_dog', 'box_turtle', 'chow', 'shopping_cart',
       'miniature_poodle', 'golden_retriever', 'Gordon_setter',
       'Walker_hound', 'pug', 'bloodhound', 'Lhasa', 'English_setter',
       'hen', 'desktop_computer', 'Italian_greyhound', 'Maltese_dog',
       'three-toed_sloth', 'ox', 'malamute', 'guinea_pig',
       'soft-coated_wheaten_terrier', 'Chihuahua',
       'black-and-tan_coonhound', 'coho', 'toy_terrier'], dtype=object)

In [29]:
# Looking for inconsistencies in p2 values
image_predictions_df.p2.unique()[:30]

array(['collie', 'miniature_pinscher', 'malinois', 'redbone',
       'Rottweiler', 'English_springer', 'mud_turtle', 'Tibetan_mastiff',
       'shopping_basket', 'komondor', 'Yorkshire_terrier',
       'English_foxhound', 'bull_mastiff', 'German_shepherd', 'Shih-Tzu',
       'Newfoundland', 'cock', 'desk', 'toy_terrier', 'toy_poodle',
       'otter', 'Chesapeake_Bay_retriever', 'Siberian_husky', 'skunk',
       'Afghan_hound', 'bloodhound', 'barracouta', 'papillon',
       'cocker_spaniel', 'chow'], dtype=object)

#### Quality issues observed
`WeRateDogs Twitter Archive`:
1. IDs columns are floats not integers  
2. Timestamp is a string not a datetime object
3. Source should be cleaned to make it more readable
4. Retweet timestamp is a string not a datetime object
5. Name column has invalid values like 'a', 'one', 'in', for example.
6. Doggo, floofer, pupper and puppo should be ints not string.
7. There's a lot of NaN values.

`Image Predictions`:
1. Some strings are lowercase and some aren't in columns p1, p2, p3.
2. There are only 2075 image predictions, 281 less than the number of tweets in the WeRateDogs Archive.

#### Tidiness issues observed
Write the tidiness issues observed

### Clean
Explain purpose of Clean

#### Define
Define how you will address each problem identified

#### Code
Solve the problems

#### Test
Test if your code worked

## Exploratory Data Analysis
Explain EDA

## Conclusion

Write conclusions